In [19]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## GET INDEKS LINK

In [20]:
def getIndeksLink(links, page, cat_link, offset=0, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY/mm/dd
    """
    print("page ", page)
    url = "https://index.sindonews.com/index/"+ str(cat_link)+ "/" + str(offset)+ "?t="+ date
    print(url)
    
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    
    contentDiv = soup.find('div', class_="indeks-news")
    if contentDiv:
        for post in contentDiv.findAll('div', class_="indeks-title"):
            link = [post.find('a', href=True)['href'], ""]
            links.append(link)
        
    el_page = soup.find('div', class_="pagination")
    if el_page:
        max_page = int(soup.find('div', class_="pagination").findAll('a')[-2]['data-ci-pagination-page'])
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, offset+10, date)
        
    return links

## GET DETAIL BERITA

In [52]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb")
        if not bc:
            continue
        cat = bc.findAll('a')[-2].text
        sub = bc.findAll('a')[-1].text
            
        #articleid
        url_split = url.replace('//','').split('/')
        article_id = url_split[2]
        articles['id'] = article_id
        
        #category
        articles['category'] = cat
        articles['subcategory'] = sub
        
        #article_url
        articles['url'] = url        

        #article
        article = soup.find("div", id="content")

        #extract date
        pubdate = soup.find('time').text
        pubdate = pubdate.strip(' \t\n\r')
        pubdate = pubdate.replace(' WIB','')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A, %d %B %Y - %H:%M"), "%Y-%m-%d %H:%M:%S") 

        #extract author
        reporter = soup.find('div', class_="reporter")
        author = reporter.find('p', class_="author").find('a').text.strip(' ')
        articles['author'] = author
        
        #extract title
        title = soup.find('div', class_="article").find('h1').text
        if ("foto" in title.lower()) or  "video" in title.lower(): 
            continue
        articles['title'] = title   

        #source
        articles['source'] = 'Sindonews'

        #extract comments count
        articles['comments'] = 0

        #extract tags
        #tags = soup.findAll('span', class_="tags--snippet__name")
        #tags = ','.join([x.text for x in tags])
        #articles['tags'] = tags

        #extract images
        image = soup.find('div', class_="article").find('img')['src']
        articles['image'] = image
        
        #hapus link sisip image
        for link in article.findAll('img'):
            link.decompose()
            
        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
            
        #articles['content'] = re.sub('google*','', content).strip(' ')
        articles['content'] = content
        #print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [23]:
links = getIndeksLink([], 1, 0, 0, '2018-07-01')

page  1
https://index.sindonews.com/index/0/0?t=2018-07-01
page  2
https://index.sindonews.com/index/0/10?t=2018-07-01
page  3
https://index.sindonews.com/index/0/20?t=2018-07-01
page  4
https://index.sindonews.com/index/0/30?t=2018-07-01
page  5
https://index.sindonews.com/index/0/40?t=2018-07-01
page  6
https://index.sindonews.com/index/0/50?t=2018-07-01
page  7
https://index.sindonews.com/index/0/60?t=2018-07-01
page  8
https://index.sindonews.com/index/0/70?t=2018-07-01
page  9
https://index.sindonews.com/index/0/80?t=2018-07-01
page  10
https://index.sindonews.com/index/0/90?t=2018-07-01
page  11
https://index.sindonews.com/index/0/100?t=2018-07-01
page  12
https://index.sindonews.com/index/0/110?t=2018-07-01
page  13
https://index.sindonews.com/index/0/120?t=2018-07-01
page  14
https://index.sindonews.com/index/0/130?t=2018-07-01
page  15
https://index.sindonews.com/index/0/140?t=2018-07-01
page  16
https://index.sindonews.com/index/0/150?t=2018-07-01
page  17
https://index.sindo

In [24]:
links

[['https://sports.sindonews.com/read/1317994/50/khan-klaim-duel-lawan-vargas-bisa-jadi-laga-eliminasi-gelar-wba-1530453589',
  ''],
 ['https://soccer.sindonews.com/pialadunia/read/1318014/135/taktik-catenaccio-bikin-spanyol-terusir-lewat-drama-adu-penalti-1530463285',
  ''],
 ['https://ekbis.sindonews.com/read/1317955/178/kolaborasi-bni-dan-ptpn-xii-dukung-petani-kopi-1530440471',
  ''],
 ['https://daerah.sindonews.com/read/1318009/21/keterbatasan-fisik-guru-ngaji-dapat-bantuan-iqra-dan-kursi-roda-1530461356',
  ''],
 ['https://nasional.sindonews.com/read/1318016/13/menurut-badan-pemeriksa-keuangan-msaa-bdni-sudah-closing-1530465801',
  ''],
 ['https://sports.sindonews.com/read/1318006/49/dominasi-race-2-herlings-dan-cairoli-saling-balas-pujian-1530459984',
  ''],
 ['https://soccer.sindonews.com/pialadunia/read/1318008/135/laga-spanyol-vs-rusia-lanjut-ke-babak-tambahan-1530460635',
  ''],
 ['https://international.sindonews.com/read/1317991/42/temui-putin-bolton-singgung-campur-tangan-r

In [53]:
details = getDetailBerita(links[0:3])
details

[{'author': 'Wahyu Nugroho',
  'category': 'Sports',
  'comments': 0,
  'content': 'BIRMINGHAM - Mantan juara tinju dunia asal Inggris, Amir "King" Khan pada September mendatang dijadwalkan naik ring lagi. Petinju 31 tahun itu akan berhadapan dengan Samuel Vargas di Arena Birmingham, Inggris.  Duel melawan petinju Kolombia-Kanada ini akan menjadi pertempuran kedua Khan sejak kembali aktif, setelah hampir dua tahun absen. Di laga comeback-nya, April lalu, Khan hanya membutuhkan waktu 39 detik untuk menghentikan Phil Lo Greco.  Menurut Khan, kontes versus Vargas berpeluang untuk menjadi eliminasi akhir gelar WBA di kelas welter. Saat ini, mahkota WBA berada dalam kekuasaan Lucas Martin Matthysse, yang dua pekan lagi akan melakoni bentrokan versus Manny Pacquiao.  "Ini akan menjadi luar biasa untuk kembali ke ring, terutama bertempur di Inggris lagi," tegas Khan saat diwawancarai Sky Sports seusai konferensi pers baru-baru ini. "Saya harap pertarungan ini berlangsung lebih lama dari yang 

In [54]:
df = pd.DataFrame(details)

In [55]:
df

,author,category,comments,content,id,image,pubdate,source,subcategory,title,url
0,Wahyu Nugroho,Sports,0,BIRMINGHAM - Mantan juara tinju dunia asal Ing...,1317994,https://cdn.sindonews.net/dyn/620/content/2018...,2018-07-01 23:50:00,Sindonews,Tinju,Khan Klaim Duel Lawan Vargas Bisa Jadi Laga El...,https://sports.sindonews.com/read/1317994/50/k...
1,Hafid Fuad,Ekbis,0,BANYUWANGI - Peran bank tidak lagi hanya seked...,1317955,https://cdn.sindonews.net/dyn/620/content/2018...,2018-07-01 23:15:00,Sindonews,Perbankan,Kolaborasi BNI dan PTPN XII Dukung Petani Kopi,https://ekbis.sindonews.com/read/1317955/178/k...
